In [41]:
import mariadb
import sys
import pandas as pd


def import_file_into_db_table_(filename, database, user, passwd, tableName,
createTableStatement = "", overwrite = False):
    try:
        connection = mariadb.connect(
        user=user,
        password=passwd, # or insert string here
        host="127.0.0.1",
        port=3306,
        database=database
        )
    except mariadb.Error as error:
        print(f'Error connecting to MariaDB: {error}')
        sys.exit(1)

    cursor = connection.cursor()

    


    table_exists = False
    try:
        cursor.execute(f"""DESCRIBE student_mat;""")
        table_exists = True
    except:
        pass

    if(table_exists):
        if(overwrite):
            cursor.execute(f"""LOAD DATA INFILE "{filename}"
                                REPLACE
                                INTO TABLE {tableName}
                                FIELDS TERMINATED BY ';'
                                ENCLOSED BY '"'
                                IGNORE 1 LINES;""")
        else:
            print('Could not write values into table, because table already exists')
            sys.exit(1)
    else:
        if createTableStatement:
            try:
                cursor.execute(createTableStatement)
            except:
                print("createTableStatement failed")
                sys.exit(1)

        cursor.execute(f"""LOAD DATA INFILE "{filename}"
                            INTO TABLE {tableName}
                            FIELDS TERMINATED BY ';'
                            ENCLOSED BY '"'
                            IGNORE 1 LINES;""")
        
    

    # check if rows match
    df = pd.read_csv(filename, delimiter=";")
    cursor.execute(f"SELECT COUNT(*) FROM {tableName}")
    if len(df.index) == cursor.fetchall()[0][0]:
        print("Length of table matches the CSV file")
    else:
        print("Length of table does not match the CSV file")


createTableStatement = ""
with open("createTableStatement.sql") as f:
    createTableStatement = f.read()

import_file_into_db_table_(filename="student-mat.csv", database="db2022", user="root", passwd="password", tableName="student_mat", createTableStatement=createTableStatement)


Could not write values into table, because table already exists


SystemExit: 1

/home/main/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
